Trying LCEL

In [1]:
pip install langchain bs4 sentence_transformers --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


In [50]:
urls = [
    "https://www.foxnews.com/sports/chiefs-patrick-mahomes-regrets-giving-bills-josh-allen-expletive-earful-loss",
    "https://www.syracuse.com/buffalo-bills/2023/12/chiefs-patrick-mahomes-expresses-regret-over-how-he-treated-bills-josh-allen-post-game.html",
    "https://athlonsports.com/nfl/travis-kelce-heartbreaking-five-word-message-josh-allen-bills-chiefs-craziest",
    "https://www.espn.com/nfl/recap/_/gameId/401547583",
    "https://apnews.com/article/offside-chiefs-bills-kelce-toney-9b62fb1c2cb9219e84faa9d59c85ddd6"
]

In [51]:
model = Ollama(model='mistral')


embedder_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embedder = HuggingFaceBgeEmbeddings(
    model_name=embedder_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


# url = "https://en.wikipedia.org/wiki/Cyclone_Michaung"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

split_data = []
for url in urls:
    text_loader = WebBaseLoader(url)

    split_data += text_splitter.split_documents(text_loader.load())


vectorstore = Chroma.from_documents(documents=split_data, embedding=embedder)

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
)

In [71]:
template = """
You are a news expert who answers questions about news.  

You must only use the most recent data from this context:
{context}

Do not rely on your historical records.

Answer as concisely as possible, but make sure that your information
lines up with the sources.  

You must name any sources at the end of your answer in the following format: 
"Sources: \n[source name, article title]"
Do NOT povide any urls.

If you don't have the relevent information to answer the question or a source,
tell the user so.  Err on the side of caution.

If the question is not about news, tell the user "Sorry, I can only answer 
questions about news".

Here is the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = setup_and_retrieval | prompt | model | output_parser

In [72]:
print(chain.invoke("Who was offsides in the Bills/Chiefs' game?"))


In the Bills/Chiefs game, Kadarius Toney was called for an offside penalty that led to a 49-yard score being called back. The penalty occurred during a pass from Patrick Mahomes to Travis Kelce.
